In [1]:
%load_ext autoreload
%autoreload 2

from gpu import set_gpu
import numpy as np
import adapt.loop as lp
import adapt.ml.dl_subclass as dl
import tensorflow as tf
from keras.utils import to_categorical
import copy as cp

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:

(x_train, y_train_lda), (x_test, y_test_lda) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(x_train.shape[0],-1)
y_train = to_categorical(y_train_lda)
x_test = x_test.reshape(x_test.shape[0],-1)
y_test = to_categorical(y_test_lda)
x_train = x_train.astype(np.float32)/ 255
x_test = x_test.astype(np.float32)/ 255

trainmlp = tf.data.Dataset.from_tensor_slices((x_train, y_train, y_train)).shuffle(x_train.shape[0],reshuffle_each_iteration=True).batch(100)

# return a new mnist dataset w/ pixels randomly permuted
def permute_mnist(mnist):
    perm_inds = list(range(mnist.shape[1]))
    np.random.shuffle(perm_inds)
    mnist2 = cp.deepcopy(mnist)
    mnist2 = np.transpose(np.array([mnist2[:,c] for c in perm_inds]))
    return mnist2

mnistb = permute_mnist(x_train)
trainmlp2 = tf.data.Dataset.from_tensor_slices((mnistb, y_train, y_train)).shuffle(x_train.shape[0],reshuffle_each_iteration=True).batch(100)


In [3]:
ep =1
mlp, _, w, c = lp.train_models(trainmlp, trainmlp, None, None, 10, ep=ep)
mlp2, _, w, c = lp.train_models(trainmlp, trainmlp2, None, None, 10, ep=ep)

In [4]:
test_ewc = dl.EWC(n_class=10)
test_ewc.call(x_train[:1,...])
# test_ewc.star(mlp)
# test_ewc.compute_diag_fim(x_train,y_train)

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.12983836, 0.10897172, 0.08585899, 0.06529689, 0.07696177,
        0.07230198, 0.08424954, 0.16209799, 0.08999179, 0.124431  ]],
      dtype=float32)>

In [5]:
ep = 30

lam = 0
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_loss2 = tf.keras.metrics.Mean(name='train_loss2')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

train_ewc = dl.get_train_ewc()

for epoch in range(ep):
    train_loss.reset_states()
    train_loss2.reset_states()
    train_accuracy.reset_states()
    for x, y, _ in trainmlp:
        train_ewc(x, y, test_ewc, optimizer, train_loss, train_loss2, train_accuracy, lam=lam)
    if epoch == 0 or epoch == ep-1:
        print(f'Epoch {epoch + 1}, ', f'Loss: {train_loss.result():.2f}, ', f'Accuracy: {train_accuracy.result() * 1:.2f} ')

Epoch 1,  Loss: 4.51,  Accuracy: 0.11 
Epoch 30,  Loss: 2.35,  Accuracy: 0.10 


In [6]:
test_mod = dl.get_test()
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')
test_mod(x_train, y_train, mlp, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, mlp, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(x_train, y_train, mlp2, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, mlp2, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(x_train, y_train, test_ewc, test_loss, test_accuracy)
print(test_accuracy.result()*100)

test_mod = dl.get_test()
test_loss.reset_states()
test_accuracy.reset_states()
test_mod(mnistb, y_train, test_ewc, test_loss, test_accuracy)
print(test_accuracy.result()*100)

tf.Tensor(93.475, shape=(), dtype=float32)
tf.Tensor(6.5933332, shape=(), dtype=float32)
tf.Tensor(11.013333, shape=(), dtype=float32)
tf.Tensor(93.041664, shape=(), dtype=float32)
tf.Tensor(9.863334, shape=(), dtype=float32)
tf.Tensor(9.863334, shape=(), dtype=float32)
